In [13]:
import time
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from basketball_reference_web_scraper import client
from datetime import date
from functools import reduce
from datetime import datetime
from collections import Counter
from dateutil.rrule import rrule, DAILY
from basketball_reference_web_scraper import client
from basketball_reference_web_scraper.data import Team
from basketball_reference_web_scraper.data import Outcome
from basketball_reference_web_scraper.data import Position

In [14]:
'''
NOTE: This section of code is *adapted* from https://github.com/stevenrdungan/mvp.
It downloads NBA MVP voting data from basketball reference using web scraping.
# Runtime 1-2 minutes
I will take this voting data and build a regression model using my leaders dataset created above to test whether
leading one's team the most per season in stats in each game is a good predictor for MVP votes.
'''

import os
from bs4 import BeautifulSoup
import requests
import csv

def get_response(url):
    print(f"Requesting \'{url}\'")
    try:
        r = requests.get(url)
        text = r.text
        return BeautifulSoup(text, 'lxml')
    except requests.exceptions.RequestException as e:
        print(e)
    return None

# create directory for data
directory = os.path.join(os.getcwd(), 'scrapedata')
print(f"directory is '{directory}'")
if not os.path.exists(directory):
    print(f"Creating directory \'{directory}\'")
    os.makedirs(directory)

# scrape player per game statistic data 1988-2019
for year in range(1988,2020):
    data = []
    url = f"http://www.basketball-reference.com/leagues/NBA_{year}_per_game.html"
    soup = get_response(url)
    table = soup.find('table', attrs={'id':'per_game_stats'})
    table_head = table.find('thead')
    hrow = table_head.find('tr')
    hcols = hrow.find_all('th')
    hcols = [ele.text.strip() for ele in hcols]
    data.append([ele for ele in hcols if ele])
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols = [ele.text.strip() for ele in row.find_all('th')]
        # ignore rows that contain header information
        if cols[0] == 'Rk':
            continue
        cols += [ele.text.strip() for ele in row.find_all('td')]
        # data.append([ele for ele in cols if ele])
        # occasionally records representing pct are blank if num and denom are zero
        data.append([ele for ele in cols])
    fname = f"{directory}/{table.get('id')}_{year}.csv"
    with open(fname, 'w') as file:
        wr = csv.writer(file)
        wr.writerows(data)

# scrape player advanced statistic data 1988-2019
for year in range(1988,2020):
    data = []
    url = f"http://www.basketball-reference.com/leagues/NBA_{year}_advanced.html"
    soup = get_response(url)
    table = soup.find('table', attrs={'id':'advanced_stats'})
    table_head = table.find('thead')
    hrow = table_head.find('tr')
    hcols = hrow.find_all('th')
    hcols = [ele.text.strip() for ele in hcols]
    data.append([ele for ele in hcols if ele])
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols = [ele.text.strip() for ele in row.find_all('th')]
        # ignore rows that contain header information
        if cols[0] == 'Rk':
            continue
        cols += [ele.text.strip() for ele in row.find_all('td')]
        data.append([ele for ele in cols if ele])
    fname = f"{directory}/{table.get('id')}_{year}.csv"
    with open(fname, 'w') as file:
        wr = csv.writer(file)
        wr.writerows(data)

#scrape MVP voting data 1988-2018
for year in range(1988,2019):
    data = []
    url = f"http://www.basketball-reference.com/awards/awards_{year}.html"
    soup = get_response(url)
    if not soup:
        continue
    table = soup.find('table', attrs={'id':'mvp'})
    table_head = table.find('thead')
    hrow = table_head.find_all('tr')[1]
    hcols = hrow.find_all('th')
    hcols = [ele.text.strip() for ele in hcols]
    data.append([ele for ele in hcols if ele])
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    # because the 'Rank' column is broken we'll do it ourselves
    rank = 0
    for row in rows:
        rank += 1
        line = [rank]
        cols = [ele.text.strip() for ele in row.find_all('td')]
        line += [ele for ele in cols]
        data.append([ele for ele in line if ele])
    fname = f"{directory}/mvp_voting_{year}.csv"
    with open(fname, 'w') as file:
        wr = csv.writer(file)
        wr.writerows(data)

# scrape standings data 1988-2019
for year in range(1988,2020):
    url = f"http://www.basketball-reference.com/leagues/NBA_{year}_standings.html"
    soup = get_response(url)
    tables = soup.find_all('table')
    for table in tables:
        tid = table.get('id')
        data = []
        table_head = table.find('thead')
        hrow = table_head.find('tr')
        hcols = hrow.find_all('th')
        hcols = [ele.text.strip() for ele in hcols]
        data.append([ele for ele in hcols if ele])
        table_body = table.find('tbody')
        rows = table_body.find_all('tr')
        for row in rows:
            cols = [ele.text.strip() for ele in row.find_all('th')]
            cols += [ele.text.strip() for ele in row.find_all('td')]
            data.append([ele for ele in cols if ele])
        fname = f"{directory}/{tid}_{year}.csv"
        with open(fname, 'w') as file:
            wr = csv.writer(file)
            wr.writerows(data)

directory is '/Users/crazyforcandy1234/Desktop/Desktop_91117/Academics/Tutorials/Data_anal/NBA_MVP_LEADERSDATA/scrapedata'
Requesting 'http://www.basketball-reference.com/leagues/NBA_1988_per_game.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_1989_per_game.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_1990_per_game.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_1991_per_game.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_1992_per_game.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_1993_per_game.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_1994_per_game.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_1995_per_game.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_1996_per_game.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_1997_per_game.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_1998_per_game.htm

Requesting 'http://www.basketball-reference.com/leagues/NBA_1997_standings.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_1998_standings.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_1999_standings.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_2000_standings.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_2001_standings.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_2002_standings.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_2003_standings.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_2004_standings.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_2005_standings.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_2006_standings.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_2007_standings.html'
Requesting 'http://www.basketball-reference.com/leagues/NBA_2008_standings.html'
Requesting 'http://www.baske

In [15]:
#Scraping top MVP candidates from the current year from https://www.basketball-reference.com/friv/mvp.html
url = f"https://www.basketball-reference.com/friv/mvp.html"
soup = get_response(url)
tables = soup.find_all('table')
for table in tables:
    tid = table.get('id')
    data = []
    table_head = table.find('thead')
    hrow = table_head.find('tr')
    hcols = hrow.find_all('th')
    hcols = [ele.text.strip() for ele in hcols]
    data.append([ele for ele in hcols if ele])
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols = [ele.text.strip() for ele in row.find_all('th')]
        cols += [ele.text.strip() for ele in row.find_all('td')]
        data.append([ele for ele in cols if ele])
df=pd.DataFrame(data)
df.columns=df.iloc[0]
df=df.drop([0]).drop(columns=['Rk'])
df['Prob%'] = df['Prob%'].str.replace('%', '').astype(float)
cols = df.columns.drop(['Player','Tm'])
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
df["Rank"]=df.index
df["Share"]=df["Prob%"]/58 #convert the given probability to match share

#create same directory
directory = os.path.join(os.getcwd(), 'scrapedata')
print(f"directory is '{directory}'")
if not os.path.exists(directory):
    print(f"Creating directory \'{directory}\'")
    os.makedirs(directory)

#format name the same
df.to_csv(f"{directory}/mvp_voting_2019.csv")


Requesting 'https://www.basketball-reference.com/friv/mvp.html'
directory is '/Users/crazyforcandy1234/Desktop/Desktop_91117/Academics/Tutorials/Data_anal/NBA_MVP_LEADERSDATA/scrapedata'


In [43]:
import os
import re

'''
NOTE: This section of code is adapted from https://github.com/stevenrdungan/mvp

Merges all our scraped data together - MVP voting data, Per game data, Seed, Advanced stats

I'll use this to make MVP predictions. AND I hope to utilize All-Star data: take data from seasons up until the all star selection and use leaders dataset
to predict the binary all-star selection.
'''

teams = {'Atlanta Hawks':'ATL',
'Boston Celtics':'BOS',
'Brooklyn Nets':'BRK',
'Charlotte Bobcats':'CHA',
'Charlotte Hornets':'CHO',
'Chicago Bulls':'CHI',
'Cleveland Cavaliers':'CLE',
'Dallas Mavericks':'DAL',
'Denver Nuggets':'DEN',
'Detroit Pistons':'DET',
'Golden State Warriors':'GSW',
'Houston Rockets':'HOU',
'Indiana Pacers':'IND',
'Los Angeles Clippers':'LAC',
'Los Angeles Lakers':'LAL',
'Memphis Grizzlies':'MEM',
'Miami Heat':'MIA',
'Milwaukee Bucks':'MIL',
'Minnesota Timberwolves':'MIN',
'New Jersey Nets':'NJN',
'New Orleans Hornets':'NOH',
'New Orleans Pelicans':'NOP',
'New OrleansOklahoma City Hornets':'NOK',
'New York Knicks':'NYK',
'Oklahoma City Thunder':'OKC',
'Orlando Magic':'ORL',
'Philadelphia 76ers':'PHI',
'Phoenix Suns':'PHO',
'Portland Trail Blazers':'POR',
'Sacramento Kings':'SAC',
'San Antonio Spurs':'SAS',
'Seattle SuperSonics':'SEA',
'Toronto Raptors':'TOR',
'Utah Jazz':'UTA',
'Vancouver Grizzlies':'VAN',
'Washington Bullets':'WSB',
'Washington Wizards':'WAS'}

pd.set_option('precision', 3)
data = pd.DataFrame()   # this will be our dataset
directory = os.path.join(os.getcwd(),'scrapedata')

for year in range(1988,2020):
    # read data into DataFrames
    pergame, advanced, voting, east, west = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    for filename in os.listdir(directory):
        if re.match(f"per_game_stats_{year}.csv", filename):
            pergame = pd.read_csv(os.path.join(directory,filename))
            if year == 2017:
                if 'PS/G' in pergame.columns:
                    pergame=pergame.rename(index=str, columns={"PS/G": "PTS"})
        elif re.match(f"advanced_stats_{year}.csv", filename):
            advanced = pd.read_csv(os.path.join(directory,filename))
        elif re.match(f"mvp_voting_{year}.csv", filename):
            voting = pd.read_csv(os.path.join(directory,filename))
        elif re.match(f"[a-z]+s_standings_E_{year}.csv", filename):
            east = pd.read_csv(os.path.join(directory,filename))
        elif re.match(f"[a-z]+s_standings_W_{year}.csv", filename):
            west = pd.read_csv(os.path.join(directory,filename))

    # assemble stats dataframe
    pergame = pergame.loc[:,['Player','Age','Tm','G','MP','TRB','AST','STL','BLK','PTS',"FGA",'FG%','3PA','3P%','eFG%','FT%','TOV','PF']]
    advanced = advanced.loc[:,['Player','Age','Tm','PER','TS%','USG%','VORP','WS','TRB%','AST%','STL%','BLK%','TOV%','WS/48','BPM']]
    if year == 2019:
        advanced["WS"]=advanced["WS"]*(82/pergame["G"].max())  #ADJUST WIN SHARE FOR FULL SEASON
        advanced["VORP"]=advanced["VORP"]*(82/pergame["G"].max())  #ADJUST WIN SHARE FOR FULL SEASON
    stats = pd.merge(pergame, advanced, on=['Player','Age','Tm'], how='left')
    stats['Year'] = year
    # remove asterisk symbol from player name (Hall of Famers)
    stats['Player'].replace(to_replace=r'\*', value=r'', regex=True, inplace=True)
    # drop all duplicate rows (i.e. players who played on multiple teams in same season)
    stats = stats.drop_duplicates(subset=['Player','Age'], keep=False)
    # only keep rows for players playing 25 minutes per game or more
    stats = stats[stats.MP >= 25.0]
    # box is sum of rebounds, assists, steals, blocks
    stats['box'] = stats['TRB'] + stats['AST'] + stats['STL'] + stats['BLK']

    # assemble standings dataframe. sort 2017 playoff teams can be easily determined
    east = east.rename(columns = {'Eastern Conference':'Tm'}).sort_values('W/L%', ascending=False).reset_index(drop=True)
    west = west.rename(columns = {'Western Conference':'Tm'}).sort_values('W/L%', ascending=False).reset_index(drop=True)
    # this will remove the Division/Conference header lines
    standings = pd.concat([east, west]).dropna()
    standings = standings.loc[:,['Tm','W','L','W/L%']]
    standings['playoffs'] = standings['Tm'].str.contains('\*').astype(int)
    standings['seed'] = standings.index + 1

    if year == 2019:
        standings['playoffs'][standings.index < 8] = 1   # assume playoffs for top 8 teams in each conference
    standings['games'] = standings['W'] + standings ['L']
    standings['Tm'] = standings['Tm'].str.replace('[^\w\s]+','').str.replace('\d+\s*$','').str.strip()
    standings = standings.replace({'Tm':teams}, regex=True)

    if year < 2003:   # if year is < 2003 replace CHA with CHH. ugly but it works!
        standings['Tm'].replace('CHO','CHH', inplace=True)
    standings = standings.drop(['W','L'], axis=1)
    df_merge = pd.merge(stats, standings, on='Tm', how='left')
    
    if not voting.empty:
        voting['Tm'] = voting['Tm'].str.strip()
        voting = voting.loc[:,['Player','Tm','Share','Rank']]
        df_merge = pd.merge(df_merge, voting, on=['Player','Tm'], how='left')
        df_merge['got_votes'] = (df_merge['Share'] > 0).astype(int)
        df_merge['is_mvp'] = (df_merge['Rank'] == 1).astype(int)
        df_merge['Share'].fillna(0, inplace=True)
    else:
        df_merge['is_mvp'] = np.nan
        df_merge['got_votes'] = np.nan
        df_merge['Share']= np.nan
    df_merge['gp_pct'] = df_merge['G'] / df_merge['games']
    df_merge = df_merge.drop(['G','games'], axis=1)
    if data.empty:
        data = df_merge
    else:
        data = pd.concat([data,df_merge])

data = data.sort_values('Share', ascending=False)
data['Share'] = data['Share'].map(lambda x: '{0:.3}'.format(x))
data['TS%'] = data['TS%'].map(lambda x: '{0:.3}'.format(x))
data['W/L%'] = data['W/L%'].map(lambda x: '{0:.3}'.format(x))
data['gp_pct'] = data['gp_pct'].map(lambda x: '{0:.3}'.format(x))
data=data.set_index('Player')
data['Share']=data.Share.astype('float')
data['TS%']=data['TS%'].astype('float')
data['W/L%']=data['W/L%'].astype('float')
data['gp_pct']=data['gp_pct'].astype('float')

all_data=data[data.columns]

# output to csv
outdir = os.path.join(os.getcwd(),'output')
if not os.path.exists(outdir):
    print(f"Creating directory \'{outdir}\'")
    os.makedirs(outdir)
all_data.to_csv(outdir + '/full_dataframe.csv', float_format='%.1f')

all_data



/Users/crazyforcandy1234/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:100: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,Age,Tm,MP,TRB,AST,STL,BLK,PTS,FGA,FG%,...,Year,box,W/L%,playoffs,seed,Share,Rank,got_votes,is_mvp,gp_pct
Player,,,,,,,,,,,,,,,,,,,,,
Stephen Curry,27,GSW,34.2,5.4,6.7,2.1,0.2,30.1,20.2,0.504,...,2016,14.4,0.890,1,1,1.000,1.0,1,1,0.963
LeBron James,28,MIA,37.9,8.0,7.3,1.7,0.9,26.8,17.8,0.565,...,2013,17.9,0.805,1,1,0.998,1.0,1,1,0.927
Shaquille O'Neal,27,LAL,40.0,13.6,3.8,0.5,3.0,29.7,21.1,0.574,...,2000,20.9,0.817,1,1,0.998,1.0,1,1,0.963
Kevin Garnett,27,MIN,39.4,13.9,5.0,1.5,2.2,24.2,19.6,0.499,...,2004,22.6,0.707,1,1,0.991,1.0,1,1,1.000
Michael Jordan,32,CHI,37.7,6.6,4.3,2.2,0.5,30.4,22.6,0.495,...,1996,13.6,0.878,1,1,0.986,1.0,1,1,1.000
Kevin Durant,25,OKC,38.5,7.4,5.5,1.3,0.7,32.0,20.8,0.503,...,2014,14.9,0.720,1,2,0.986,1.0,1,1,0.988
LeBron James,25,CLE,39.0,7.3,8.6,1.6,1.0,29.7,20.1,0.503,...,2010,18.5,0.744,1,1,0.980,1.0,1,1,0.927
Derrick Rose,22,CHI,37.4,4.1,7.7,1.0,0.6,25.0,19.7,0.445,...,2011,13.4,0.756,1,1,0.977,1.0,1,1,0.988
LeBron James,24,CLE,37.7,7.6,7.2,1.7,1.1,28.4,19.9,0.489,...,2009,17.6,0.805,1,1,0.969,1.0,1,1,0.988


In [86]:
"""
THIS DATASET HAS 31 YEARS OF PLAYER STATS
(CURRENTLY FOR ALL NBA PLAYERS WHO PLAYED AN AVERAGE OF 25 MINUTES PER GAME, ~ TOP 150 PLAYERS FOR MINS PLAYED PER GAME EACH SEASON)
THE STATS INCLUDED ARE BOTH PER-GAME STATS, AS IN HOW MANY POINTS OR ASSISTS THE PLAYER SCORED ON AVERAGE PER GAME,
AS WELL AS ADVANCED STATS LIKE WIN SHARE (WS) AND VORP (VALUE OVER REPLACEMENT).
ALSO INCLUDED ARE THE PLAYER'S TEAM STATS FOR THAT SEASON: WIN/LOSE % AND SEED.
FINALLY, WE HAVE 3 VARIABLES OF MVP DATA: IS_MVP (BINARY, 1 IF MVP), GOT_VOTES (1 IF GOT VOTES), AND SHARE (VOTE SHARE)

I WILL TRAIN VARIOUS COMBINATIONS OF PERGAME, ADVANCED, TEAM STATS TO PREDICT THE 3 MVP OUTCOME VARIABLES.

I'LL TAKE 2019 OUT OF THE TRAIN DATA BECAUSE THERE IS NO OUTCOME YET.

BEFORE WE START ANY REGRESSIONS A GOOD WAY TO KICK OFF IS BY LOOKING AT CORRELATIONS BETWEEN EACH VARIABLE & VOTE SHARE

I WILL ADD OTHER NBA AWARD DATA, AS WELL AS ALL-STAR SELECTION

WILL CONDUCT RDD ANALYSIS FOR ALL PLAYERS THAT GOT VOTES WITH IS_MVP AS THE CUTOFF VALUES

"""

Index(['Player', 'Age', 'Tm', 'PER', 'TS%', 'USG%', 'VORP', 'WS'], dtype='object')

In [44]:
# Subset df 2000-2019 which includes player leader data (can only access 2000-2019 with bball reference web scraper)

directory = os.path.join(os.getcwd(),'season_leaders')
leaders = [pd.read_csv(f'./season_leaders/season_leaders_{year}.csv') for year in range(2000,2020)]
year=2000
for df in leaders:
    df["year"]=year
    year+=1
leaders=pd.concat(leaders).set_index('player').rename(index=str, columns={"year": "Year"})
leaders.index=leaders.index.rename("Player")
leaders["pra"]=leaders["point_lead_count_wins"]+leaders["rebound_lead_count_wins"]+leaders["assist_lead_count_wins"]
leaders["total_pra"]=leaders["point_lead_count_total"]+leaders["rebound_lead_count_total"]+leaders["rebound_lead_count_total"]
leaders["total_prank"]=leaders["total_pra"].rank(ascending=False)
leaders["pra_rank"] = leaders["pra"].rank(ascending=False) 

all_data_2000 = all_data[all_data["Year"]>=2000]
all_data_2000 = all_data_2000.merge(leaders,how='inner',on=['Player','Year'])

# output to csv
outdir = os.path.join(os.getcwd(),'output')
if not os.path.exists(outdir):
    print(f"Creating directory \'{outdir}\'")
    os.makedirs(outdir)
all_data_2000.to_csv(outdir + '/full_df_leaderdata.csv', float_format='%.1f')

all_data_2000


,Age,Tm,MP,TRB,AST,STL,BLK,PTS,FGA,FG%,...,point_lead_count_losses,rebound_lead_count_losses,assist_lead_count_losses,point_lead_count_total,rebound_lead_count_total,assist_lead_count_total,pra,total_pra,total_prank,pra_rank
Player,,,,,,,,,,,,,,,,,,,,,
Stephen Curry,27,GSW,34.2,5.4,6.7,2.1,0.2,30.1,20.2,0.504,...,6.0,0.0,3.0,55.0,33.0,9.0,88.0,121.0,208.0,20.0
LeBron James,28,MIA,37.9,8.0,7.3,1.7,0.9,26.8,17.8,0.565,...,12.0,9.0,11.0,53.0,55.0,44.0,120.0,163.0,50.0,3.0
Shaquille O'Neal,27,LAL,40.0,13.6,3.8,0.5,3.0,29.7,21.1,0.574,...,9.0,10.0,3.0,59.0,22.0,70.0,129.0,103.0,326.0,1.0
Kevin Garnett,27,MIN,39.4,13.9,5.0,1.5,2.2,24.2,19.6,0.499,...,17.0,21.0,10.0,50.0,26.0,74.0,102.0,102.0,334.5,11.5
Kevin Durant,25,OKC,38.5,7.4,5.5,1.3,0.7,32.0,20.8,0.503,...,19.0,6.0,10.0,69.0,34.0,22.0,90.0,137.0,119.0,16.5
LeBron James,25,CLE,39.0,7.3,8.6,1.6,1.0,29.7,20.1,0.503,...,16.0,6.0,14.0,68.0,64.0,23.0,119.0,196.0,7.0,4.0
Derrick Rose,22,CHI,37.4,4.1,7.7,1.0,0.6,25.0,19.7,0.445,...,12.0,0.0,16.0,54.0,67.0,2.0,95.0,188.0,14.5,14.5
LeBron James,24,CLE,37.7,7.6,7.2,1.7,1.1,28.4,19.9,0.489,...,11.0,5.0,11.0,63.0,59.0,26.0,121.0,181.0,22.0,2.0
James Harden,28,HOU,35.4,5.4,8.8,1.8,0.7,30.4,20.1,0.449,...,10.0,0.0,12.0,54.0,51.0,7.0,90.0,156.0,66.5,16.5
